# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace/dm_apache_cassandra


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
print(len(full_data_rows_list))
print(full_data_rows_list[0])

8056
['Stephen Lynch', 'Logged In', 'Jayden', 'M', '0', 'Bell', '182.85669', 'free', 'Dallas-Fort Worth-Arlington, TX', 'PUT', 'NextSong', '1.54099E+12', '829', "Jim Henson's Dead", '200', '1.54354E+12', '91']


In [5]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [6]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])
    

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [7]:
#Create a Keyspace 
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS events
    WITH REPLICATION = {'class':'SimpleStrategy',
                        'replication_factor':1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [8]:
#Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('events')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4
    






In [9]:
df = pd.read_csv('event_datafile_new.csv')
df.head()

artist firstName gender  itemInSession lastName     length level  \
0  Stephen Lynch    Jayden      M              0     Bell  182.85669  free   
1        Manowar     Jacob      M              0    Klein  247.56200  paid   
2      Morcheeba     Jacob      M              1    Klein  257.41016  paid   
3       Maroon 5     Jacob      M              2    Klein  231.23546  paid   
4          Train     Jacob      M              3    Klein  216.76363  paid   

                              location  sessionId  \
0      Dallas-Fort Worth-Arlington, TX        829   
1  Tampa-St. Petersburg-Clearwater, FL       1049   
2  Tampa-St. Petersburg-Clearwater, FL       1049   
3  Tampa-St. Petersburg-Clearwater, FL       1049   
4  Tampa-St. Petersburg-Clearwater, FL       1049   

                                   song  userId  
0                     Jim Henson's Dead      91  
1                           Shell Shock      73  
2  Women Lose Weight (Feat: Slick Rick)      73  
3             Won't Go Home Without You      73  
4                      Hey_ Soul Sister      73

In [10]:
## Give the artist, song title and song's length in the music app history that was heard during \
## sessionId = 338, and itemInSession = 4
song_playlist_session_df = df[['artist','song','length','itemInSession','sessionId']]
song_playlist_session_data = song_playlist_session_df.values.tolist()
song_playlist_session = """CREATE TABLE IF NOT EXISTS song_playlist_session(artist TEXT, song TEXT, length FLOAT, itemInSession INT, sessionId INT,
               PRIMARY KEY(sessionId, itemInSession))
"""
try:
    session.execute(song_playlist_session)
except Exception as e:
    print(e)
#import data into database one
song_playlist_session_import = """INSERT INTO song_playlist_session(artist, song, length, itemInSession, sessionId)
                VALUES (%s, %s, %s, %s, %s)
"""
for each in song_playlist_session_data:
    session.execute(song_playlist_session_import,(each[0],each[1],each[2],each[3],each[4]))


#### Do a SELECT to verify that the data have been inserted into each table

In [11]:
##Add in the SELECT statement to verify the data was entered into the table

try:
    validate_one = session.execute('SELECT artist, song, length FROM song_playlist_session WHERE sessionId = 338 AND itemInSession = 4;')
except Exception as e:
    print(e)
    
for each in validate_one:
    print(each.artist,',',each.song,',',each.length)

Faithless , Music Matters (Mark Knight Dub) , 495.30731201171875


In lecture, the lady said Apache cassandra should give every single column a table, so we create a subtable including artist, song, length, item in session ans session ID as sub-dataset of query one. And convery every single row to a list, finally import the list to the dataset.

### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [12]:
## Give only the following: name of artist, song (sorted by itemInSession) and user (first and last name)\
## for userid = 10, sessionid = 182
user_item_in_session_df = df[['artist','song','firstName','lastName','sessionId','itemInSession','userId']]
user_item_in_session_data = user_item_in_session_df.values.tolist()
user_item_in_session = """CREATE TABLE IF NOT EXISTS user_item_in_session(artist TEXT, song TEXT, firstName TEXT, lastName TEXT, sessionId INT, itemInSession INT, userId INT,
               PRIMARY KEY((userId, sessionId), itemInSession))
"""
try:
    session.execute(user_item_in_session)
except Exception as e:
    print(e)


user_item_in_session_import = """INSERT INTO user_item_in_session(artist, song, firstName, lastName, sessionId, itemInSession, userId)
                VALUES (%s, %s, %s, %s, %s, %s, %s)
"""
for each in user_item_in_session_data:
    try:
        session.execute(user_item_in_session_import,(each[0],each[1],each[2],each[3],each[4],each[5],each[6]))
    except Exception as e:
        print(e)


                    

In [13]:
try:
    validate_two = session.execute('SELECT artist, song, firstName, lastName FROM user_item_in_session WHERE userId = 10 AND sessionId = 182')
except Exception as e:
    print(e)
    
for each in validate_two:
    print(each.artist,',', each.song,',', each.firstname,each.lastname)

Down To The Bone , Keep On Keepin' On , Sylvie Cruz
Three Drives , Greece 2000 , Sylvie Cruz
Sebastien Tellier , Kilometer , Sylvie Cruz
Lonnie Gordon , Catch You Baby (Steve Pitron & Max Sanna Radio Edit) , Sylvie Cruz


The second query's data modeling strategy is almost like the first one, but the requirement has changed, so primary key should be changed, and the requirement said the result should be sorted, we set item in session as clustering column.

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [14]:
##Give every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
name_list_df = df[['firstName','lastName','userId','song']]
name_list_data = name_list_df.values.tolist()
name_list = """CREATE TABLE IF NOT EXISTS name_list(firstName TEXT, lastName TEXT, userId INT, song TEXT,
               PRIMARY KEY(song,userId));
"""
try:
    session.execute(name_list)
except Exception as e:
    print(e)

name_list_import = """INSERT INTO name_list(firstName, lastName, userId, song)
                VALUES (%s, %s, %s, %s)
"""
for each in name_list_data:
    try:
        session.execute(name_list_import,(each[0],each[1],each[2],each[3]))
    except Exception as e:
        print(e)
                    

In [15]:
try:
    validate_three = session.execute("""SELECT firstName, lastName FROM name_list WHERE song = 'All Hands Against His Own';""")
except Exception as e:
    print(e)
    
for each in validate_three:
    print(each.firstname,each.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


The third query has not mentioned user ID, but comparing with use every single keys in the third table as primary key, we can use user ID instead of first name and last name.

### Drop the tables before closing out the sessions

In [16]:
tables = ['song_playlist_session','user_item_in_session','name_list']
for each in tables:
    rows = session.execute('DROP TABLE IF EXISTS ' + str(each) + ';')

### Close the session and cluster connection¶

In [109]:
session.shutdown()
cluster.shutdown()